# RAG Hands-on Tutorial

Dieses Notebook demonstriert die Grundstruktur eines Retrieval-Augmented Generation (RAG) Systems. Dabei wird ein lokales Dokument (z. B. eine PDF-Datei) hochgeladen, in kleinere Abschnitte aufgeteilt, in eine Vektordatenbank eingebettet und anschließend mithilfe eines Sprachmodells abgefragt.

Das Ziel ist es, Fragen zu benutzerdefinierten Dokumenten zu beantworten, ohne das Modell neu trainieren zu müssen.

In [1]:
# Installieren aller nötigen packages
!pip install -q pymupdf langchain faiss-cpu litellm

In [5]:
# --- Imports & API-Key-Setup ---
import os
import fitz  # PyMuPDF
import tempfile
import litellm

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
# Definieren des OpenAI API Keys
os.environ["OPENAI_API_KEY"] = "sk-..."

### Embedden der Paper

Zuerst werden wir nun die PDF Dokumente mittels des Python packages fitz in Text umwandeln.

In [3]:
# --- Aufgabe: Lade alle PDF-Dateien aus einem Ordner und extrahiere den Text ---

def extract_text_from_pdf_folder(pdf_folder_path):
    all_text = ""
    
    # TODO: Iteriere über alle Dateien im angegebenen Ordner
        # TODO: Verarbeite nur .pdf-Dateien
        # TODO: Öffne die PDF-Datei mit fitz.open()
        # TODO: Iteriere über alle Seiten und extrahiere den Text mit .get_text()
    
    return all_text

# --- Aufruf ---
pdf_folder = "..."  # TODO: Ordnerpfad angeben
raw_text = extract_text_from_pdf_folder(pdf_folder)

# Vorschau des extrahierten Texts
print(raw_text[1000]) 


Loaded text from folder './pdfs', showing preview:

Polymer Solubility Prediction Using Large
Language Models
Published as part of ACS Materials Letters special issue “Machine Learning for Materials Chemistry”.
Sakshi Agarwal, Akhlak Mahmood, and Rampi Ramprasad*
Cite This: ACS Materials Lett. 2025, 7, 2017−2023
Read Online
ACCESS
Metrics & More
Article Recommendations
*
sı
Supporting Information
ABSTRACT: Traditional approaches in polymer informatics often require labor-intensive data curation, time-
consuming preprocessing such as fingerprinting, and choosing suitable learning algorithms. Large language models
(LLMs) represent a compelling alternative by addressing these limitations with their inherent flexibility, ease of use,
and scalability. In this study, we propose a novel approach utilizing fine-tuned LLMs to classify solvents and
nonsolvents for polymers, a property critical to polymer synthesis, purification, and diverse applications. Our results
show that fine-tuned GPT-3.5 

Als nächstes wird nun der Text in kleinere Abschnitte (engl. Chunks) aufgeteilt. Dazu müssen passende Parameter für die Länge der Chunks sowie für die Überlappung dieser gewählt werden. 

In [4]:
# --- Aufgabe: Text in überlappende Chunks aufteilen ---

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=___,       # TODO: Wähle sinnvolle Chunk-Größe
    chunk_overlap=___     # TODO: Bestimme Überlappung
)

chunks = text_splitter.split_text()  # TODO: Wähle Eingabetext
print(f"{len(chunks)} chunks created.")

260 chunks created.


Nun müssen alle erstellten Chunks in einen Vektor (sogenannte Embedding) umgewandlet werden. Die Embeddings werden mit der OpenAI API erstellt. 

In [ ]:
# Beispielkonfiguration (OpenAI, austauschbar)
litellm_model = "openai/embedding-3-small"

# Liste von Embeddings vorbereiten
embeddings = []

for chunk in chunks:
    response = litellm.completion( 
        model=litellm_model,
        input=chunk,
        api_type="embedding"
    )
    embeddings.append(response["data"][0]["embedding"]) 

print(f"{len(embeddings)} Embeddings erstellt.")

### Implementation des Vektorstores

Nun müssen die Embeddings der einzelnen Cgunks in einem Vektortore gespeichert werden. 